In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import os
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import xgboost
from pipeline_functions import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import optuna
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import ipdb

c:\Users\32mor\anaconda3\envs\finance_scoring\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
os.getcwd()

'c:\\Users\\32mor\\OneDrive\\Documents\\HEC\\M2 DSB\\introduction to finance\\Fintech_Project'

In [10]:
dirname = os.path.dirname
root = os.getcwd()
data_path = os.path.join(dirname(root),"Fintech_Project", 'data')
path = os.path.join(data_path,"PastLoans.csv")
path_new_set = os.path.join(data_path,"NewApplications_3_Round1.csv")
path_output = os.path.join(root + '/data', 'default_predictions.csv')

In [32]:
df_past_results = pd.read_csv(os.path.join(data_path,"profit_31.csv"))
df_backtesting = pd.read_csv(os.path.join(data_path, "default_predictions_backtesting.csv"))
df_backtest_xgb = pd.read_csv(os.path.join(data_path, "predictions_xgb.csv")).rename(columns={"rate":"xgb_rate"})

In [34]:
df_back_test = df_past_results.merge(df_backtesting, on='id',how="inner").merge(df_backtest_xgb, on="id",how="inner")

In [35]:
df_back_test

,id,own,competing1,competing2,borrowertype,accepted,default,profit,rate,xgb_rate
0,100001,0.174741,0.108844,NaN,3,0,0,NaN,0.863115,0.815783
1,100002,0.093990,0.198182,0.469120,1,1,0,939.9026,0.824050,0.459014
2,100003,0.104005,0.016279,0.099437,3,0,0,NaN,0.811111,0.350154
3,100004,0.218949,0.173515,0.418875,3,0,0,NaN,0.987941,1.007206
4,100005,0.145152,0.157499,0.514527,1,0,0,NaN,0.896981,0.641286
...,...,...,...,...,...,...,...,...,...,...
99995,199996,0.521431,0.235013,NaN,3,0,0,NaN,1.225719,1.884556
99996,199997,0.863647,NaN,0.417800,3,0,1,NaN,1.365347,3.679644
99997,199998,0.580397,0.542129,0.951939,3,0,1,NaN,1.251698,2.306067
99998,199999,0.551198,0.203375,NaN,2,0,0,NaN,1.238719,2.246650


In [49]:
df_back_test["winner"] = df_back_test[["competing1","competing2","xgb_rate"]].min(axis=1)

In [50]:
df_back_test[df_back_test["winner"]==df_back_test["xgb_rate"]]["profit"].sum()

-1770537.4456000002

In [41]:
df

,id,own,competing1,competing2,borrowertype,accepted,default,profit,rate,xgb_rate,winner
0,100001,0.174741,0.108844,NaN,3,0,0,NaN,0.863115,0.815783,0.108844
1,100002,0.093990,0.198182,0.469120,1,1,0,939.9026,0.824050,0.459014,0.093990
2,100003,0.104005,0.016279,0.099437,3,0,0,NaN,0.811111,0.350154,0.016279
3,100004,0.218949,0.173515,0.418875,3,0,0,NaN,0.987941,1.007206,0.173515
4,100005,0.145152,0.157499,0.514527,1,0,0,NaN,0.896981,0.641286,0.145152
...,...,...,...,...,...,...,...,...,...,...,...
99995,199996,0.521431,0.235013,NaN,3,0,0,NaN,1.225719,1.884556,0.235013
99996,199997,0.863647,NaN,0.417800,3,0,1,NaN,1.365347,3.679644,0.417800
99997,199998,0.580397,0.542129,0.951939,3,0,1,NaN,1.251698,2.306067,0.542129
99998,199999,0.551198,0.203375,NaN,2,0,0,NaN,1.238719,2.246650,0.203375
